In [1]:
import pandas as pd
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedGroupKFold, cross_val_predict
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [2]:
def load(clinical_path, gene_expression_path):
    clinical = pd.read_parquet(clinical_path)
    gene_expression = pd.read_parquet(gene_expression_path)
    
    clinical.set_index("sampleID", inplace=True)
    gene_expression.set_index("caseID", inplace=True)
    
    gene_expression.index = gene_expression.index.str.split("-").str[:4].str.join("-")

    if not clinical.index.is_unique:
        raise ValueError
    
    if not gene_expression.index.is_unique:
        raise ValueError

    common_case = clinical.index.intersection(gene_expression.index)

    clinical = clinical.loc[common_case]
    gene_expression = gene_expression.loc[common_case]

    data = pd.concat({"clinical": clinical, "gene_expression": gene_expression}, axis=1)
    data.index.name = "caseID"

    return data

In [3]:
data = load("../data/label.parquet", "../data/mRNA.omics.parquet")

data = data.loc[
    data["clinical", "patient"].notnull() &
    data["clinical", "cancer_type"].notnull() &
    data["gene_expression"].notnull().all(axis=1) &
    (data["clinical", "cancer_type"] != "Normal")
]

In [4]:
X = data["gene_expression"]
y = data["clinical"]["cancer_type"]
g = data["clinical"]["patient"]

In [5]:
cv = StratifiedGroupKFold(5, shuffle=True, random_state=0)

select_k_best = SelectKBest(f_classif, k=1000)

scaler = StandardScaler()

classifier = LogisticRegression(
    solver="lbfgs",
    multi_class="multinomial",
    max_iter=2000,
    tol=1e-2,
    class_weight="balanced",
    n_jobs=8,
    random_state=0,
)

model = make_pipeline(select_k_best, scaler, classifier)

y_pred = cross_val_predict(model, X, y, groups=g, cv=cv)

report = metrics.classification_report(y, y_pred)
print(report)